# Fairy circle formation by H<sub>2</sub>S inhibition

# Plant-sulfide model

The model describes the dynamics of a plant $P$ and sulfide $S$, using the following differential equations:

$$
\begin{aligned}
\frac{\partial P}{\partial t}&= r P (1 - \frac{P}{K}) - c P S + D_{p} \Delta P\\
\frac{\partial S}{\partial t}&= \xi\Big[\varepsilon P - d \frac{k_{s}}{P + k_{s}} S\Big] + D_{s} \Delta S
\end{aligned}
$$

Here, $r$ is the plant maximal growth rate, $K$ is the plant's carrying capcity, $c$ is the maximal death rate caused by sulfide, $k_{S}$ is the plant biomass at which sulfide toxification is half maximal, $\varepsilon$ is the sulfide prodction increase for every unit of plant growth, and $d$ is the detoxification rate.  $\xi$ is a non-negative dimensionless
parameter that quantifies the difference between
orders of magnitude for the time scales of plant ($P$) and sulfide ($S$).

In [1]:
from __future__ import absolute_import, print_function
import time
import numpy as np
import pyopencl as cl
import matplotlib.pyplot as plt
from scipy import signal
from matplotlib import rcParams
import matplotlib.animation as animation

# Widening the screen
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

rcParams['figure.dpi'] = 300
rcParams['font.size'] = 16
rcParams['font.family'] = 'StixGeneral'
plt.rc('font', size=16)          # controls default text sizes
plt.rc('axes', titlesize=16)     # fontsize of the axes title
plt.rc('axes', labelsize=16)     # fontsize of the x and y labels
plt.rc('xtick', labelsize=16)    # fontsize of the tick labels
plt.rc('ytick', labelsize=16)    # fontsize of the tick labels
plt.rc('legend', fontsize=16)    # legend fontsize
plt.rc('figure', titlesize=16)   # fontsize of the figure title

### Parameter definition 

In [2]:
# # parameters
Dp  = 0.002
Ds  = Dp/10.0
K   = 10
r   = 0.8
c   = 10.0
d   = 0.5
eps = 0.004
ks  = 0.2
xi  = 0.1

### Simulation settings

In [3]:
length    = 50.0      # 2.5 Length of the physical landscape
n         = 512       # Size of the 2D grid
endtime   = 500.0     # end time
dT        = 0.05      # calculate time step
nplot     = 300
plottime  = endtime/nplot
nsteps    = np.ceil(plottime/dT).astype(int)    #number of time steps

### Precalculations

In [4]:
dX = 2*length/n      # Spatial step size
dY = 2*length/n      # Spatial step size
Grid_Width  = n
Grid_Height = n
# %% Reporting in the simulation on the console
print(" Current grid dimensions: %d x %d cells\n" % (Grid_Width, Grid_Height));

 Current grid dimensions: 512 x 512 cells



### Defining the device that is used

In [5]:
# Setting up the OpenCL context
DeviceNr = 1   # 0 = GTX 960M
platform = cl.get_platforms()
Devices  = platform[0].get_devices()  # 0 = GPU
context  = cl.Context([Devices[DeviceNr]])
queue    = cl.CommandQueue(context)
mf       = cl.mem_flags # Memory flags are set
print(" Compute Device: %s\n" % Devices[DeviceNr].name)

 Compute Device: AMD Radeon Pro Vega 64X Compute Engine



### Initial values

In [6]:
# The array is allocated on the GPU and the initial values are copied onto it
S_host   = (1.0*(np.random.rand(n*n)<0.00)).astype(np.float32)
S_g      = cl.Buffer(context, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=S_host)

# generate several seedlings 
im = np.zeros((n, n))
#ix,iy=np.random.randint(0,n,10),np.random.randint(0,n,10)
#im[ix,iy]=1.0
im[256,156]=1
im[176,306]=1
im[336,306]=1
P_host=(im.reshape(n*n)).astype(np.float32)
#P_host   =  (1.0*(np.random.rand(n*n)<0.001)).astype(np.float32)
P_g      = cl.Buffer(context, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=P_host)

### Loading the spatial functions from the ipython file

In [7]:
with open('SpatialFunctions_iPy.cl', 'r') as myfile:
   SpatialFunctions = myfile.read()

In [8]:
# List of parameters for the OpenCL kernel. Seperate with comma without spaces
# Setting up the parameters for the Kernel
PassVars="K,Dp,Ds,r,c,d,eps,xi,ks,dX,dY,dT,Grid_Width,Grid_Height"

PassVals=eval(PassVars)
PassVars=PassVars.split(',')
Params=""

for ii in range(len(PassVals)):
    Params = Params+"#define " + PassVars[ii] + " " + str(PassVals[ii]) + " \n"

### Defining the OpenCL simulation kernel

In [9]:
ComputeCode = """

///////////////////////////////////////////////////////////////////////////////
// Simulation kernel
///////////////////////////////////////////////////////////////////////////////   

__kernel void SimulationKernel (__global float* P, __global float* S)
{

    size_t current  = get_global_id(0);
    size_t row      = floor((float)current/(float)Grid_Width);
    size_t column   = current%Grid_Width;

        if (row > 0 && row < Grid_Width-1 && column > 0 && column < Grid_Height-1)
            {                
             float dPdt = r*(1 - P[current]/K)*P[current] - c*P[current]*S[current] + Dp*d2_dxy2(P);
             float dSdt = xi*(eps*P[current] - d*ks*S[current]/(P[current]+ks)) + Ds*d2_dxy2(S);
             
             P[current] = P[current] + dPdt*dT;
             S[current] = S[current] + dSdt*dT;
            }
            
        // HANDLE Boundaries
        else 
            {
             PeriodicBoundaries(P);
             PeriodicBoundaries(S);
            }

} // End SimulationKernel
"""

### Here the kernel is compiled

In [10]:
program = cl.Program(context, Params + SpatialFunctions + ComputeCode).build()

In [11]:
from ipywidgets import FloatProgress
from IPython.display import display

# Setting up a progress bar for the simulation
print("Progress :");
PB = FloatProgress(min=0, max=nplot); display(PB) 

Ps=np.zeros((Grid_Width, Grid_Height, nplot))
Ss=np.zeros((Grid_Width, Grid_Height, nplot))

# Set up simulation parameters
global_size = P_host.shape

# Start the timer:
start_time = time.time()

# Starting the loop
for ii in range(nplot):
    # The simulation
    # We store the state of the system for <NumPlot> different times.
    Ps[:,:,ii] = P_host.reshape(Grid_Width, Grid_Height)
    Ss[:,:,ii] = S_host.reshape(Grid_Width, Grid_Height)
    for jj in range(nsteps):      
        program.SimulationKernel(queue, global_size, None, P_g, S_g)
    # Get the data from the GPU
    cl.enqueue_copy(queue, P_host, P_g)
    cl.enqueue_copy(queue, S_host, S_g)
    
    PB.value += 1 # signal to increment the progress bar
# Determining the time that we used for the simulation
elapsed_time = time.time() - start_time    
print(" Simulation took      : %1.1f (s)" % (elapsed_time))

Progress :


FloatProgress(value=0.0, max=300.0)

 Simulation took      : 4.8 (s)


### Plotting the results

In [12]:
fig, ax = plt.subplots(1, 2, figsize=(12, 5))

im0 = ax[0].imshow(P_host.reshape(n,n),cmap='YlGn',extent=[0,length,0,length], clim=(0,K));
plt.colorbar(im0, ax=ax[0],fraction=0.046,pad=0.04);ax[0].set_title('Plant biomass g/m$^2$');

im1 = ax[1].imshow(S_host.reshape(n,n),cmap='YlGn',extent=[0,length,0,length], clim=(0,0.1));
plt.colorbar(im1, ax=ax[1],fraction=0.046,pad=0.04);ax[1].set_title('H$_2$S concentration $\mu$M');

text=fig.suptitle("Time: %1.0f of %1.0f days" % (endtime, endtime), x=0.5, y=0.05, fontsize=16);

### Making a movie

In [13]:
from matplotlib import animation, rc
import os
if not os.path.exists("oldSim"):
    os.makedirs("oldSim")
    
def updatefig(i): # To update the image at each iteration
    global Ps, Ss
    im0.set_array(Ps[:,:,i])
    im1.set_array(Ss[:,:,i])
    text.set_text("Time: %1.0f of %1.0f days" % ((i+1)/nplot*endtime, endtime));
    fig.savefig('oldSim/Sulfide'+str(i)+'.pdf',bbox_inches='tight')
    return im0,im1,
#for ii in range(1,nplot):
#    updatefig(ii)
    
ani = animation.FuncAnimation(fig, updatefig, interval=100, frames = nplot, repeat=False, blit=True)
print("Save figure finshes!")

Save figure finshes!


In [14]:
from IPython.display import HTML
HTML(ani.to_html5_video())

In [15]:
Writer = animation.writers['ffmpeg']
writer = Writer(fps=10, bitrate=1800)
ani.save('S1_Sulfide.mp4', writer=writer)
print("Save movies finshes!")

Save movies finshes!


The End, Kang